In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

### Food101 Dataset Downloading and Preprocessing

In [ ]:
(train_data, valid_data) = tfds.load("food101", split=['train', 'validation'], shuffle_files=True, as_supervised=True, data_dir='/content')

In [ ]:
# learning dataset shape
train_data

<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
# Creating preprocessing functions and mapping them to dataset
def normalize_img(image, label):
  return tf.cast(image, tf.float32) / 255., label

def resize_img(image, label):
  return tf.image.resize(image, [228, 228]), label

train_data = train_data.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

train_data = train_data.map(
    resize_img, num_parallel_calls=tf.data.AUTOTUNE)


valid_data = valid_data.map(
    normalize_img, num_parallel_calls=tf.data.AUTOTUNE)

valid_data = valid_data.map(
    resize_img, num_parallel_calls=tf.data.AUTOTUNE)

# Batching the data
train_data = train_data.batch(32)
valid_data = valid_data.batch(32)
train_data = train_data.prefetch(tf.data.AUTOTUNE)
valid_data = valid_data.prefetch(tf.data.AUTOTUNE)

In [ ]:
# Preprocessed dataset shape
train_data

<PrefetchDataset element_spec=(TensorSpec(shape=(None, 228, 228, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int64, name=None))>

### Loading InceptionV3 Model and Adding Classification Layer

In [ ]:
# Loading pretrained InceptionV3
img_shape = (228, 228, 3)

pre_trained = tf.keras.applications.inception_v3.InceptionV3(weights='imagenet', include_top=False, input_shape=img_shape, pooling='avg')

for layer in pre_trained.layers:
    layer.trainable = False

87910968/87910968 [==============================] - 3s 0us/step


In [ ]:
# Constructing classification layers
x = pre_trained.output
x = tf.keras.layers.BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001)(x)
x = tf.keras.layers.Dropout(0.2)(x)
predictions = tf.keras.layers.Dense(101, activation='softmax')(x)

# Saving model weights each epoch and early stopping in case of lack of improvement
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='training_checkpoints/ckpt_{epoch}',
        save_weights_only=True),
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=10,
        verbose=2,
        restore_best_weights=True),
]
# Setting model inputs and outputs, and defining optimizer, loss, and metrics
model = tf.keras.Model(inputs = pre_trained.input, outputs = predictions)
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=0.001), loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Since we had to trained model for several iterations (due to Colab limitations), we load previous model weights before final
model.load_weights('/content/training_checkpoints/ckpt_14')

### Model Training and Saving Best Weights

Only last 10 epochs can be shown, since model was trained in several sessions (due to long training time and Colab limits). Overall number of epochs: ~30. Weights were saved automatically each epoch.

In [ ]:
STEP_SIZE_TRAIN = len(train_data)
STEP_SIZE_VALID = len(valid_data)
print(STEP_SIZE_TRAIN)

history = model.fit(train_data,
                    steps_per_epoch = STEP_SIZE_TRAIN,
                    validation_data = valid_data,
                    validation_steps = STEP_SIZE_VALID,
                    callbacks=callbacks,
                    epochs = 10,
                    verbose = 1)

2368
Epoch 1/10
2368/2368 [==============================] - 311s 130ms/step - loss: 1.4931 - accuracy: 0.6033 - val_loss: 1.6293 - val_accuracy: 0.5787
Epoch 2/10
2368/2368 [==============================] - 308s 130ms/step - loss: 1.4900 - accuracy: 0.6053 - val_loss: 1.6228 - val_accuracy: 0.5803
Epoch 3/10
2368/2368 [==============================] - 306s 129ms/step - loss: 1.4803 - accuracy: 0.6063 - val_loss: 1.6310 - val_accuracy: 0.5793
Epoch 4/10
2368/2368 [==============================] - 294s 124ms/step - loss: 1.4674 - accuracy: 0.6070 - val_loss: 1.6262 - val_accuracy: 0.5807
Epoch 5/10
2368/2368 [==============================] - 295s 124ms/step - loss: 1.4683 - accuracy: 0.6070 - val_loss: 1.6286 - val_accuracy: 0.5817
Epoch 6/10
2368/2368 [==============================] - 308s 130ms/step - loss: 1.4605 - accuracy: 0.6108 - val_loss: 1.6330 - val_accuracy: 0.5798
Epoch 7/10
2368/2368 [==============================] - 298s 126ms/step - loss: 1.4563 - accuracy: 0.6114 -

#### Best result was produced on 5th epoch of the last training session.